# WeatherPy
----

#### Analysis
Observation:  Northern Hemisphere - Max Temp vs. Latitude Linear Regression
Negative linear relationship – The temperature decreases as we move further away from the equator and latitude increases.

Observation: Southern Hemisphere - Max Temp vs. Latitude Linear Regression
Positive linear relationship – The temperature increases as we move towards the equator and latitude increases.

Observation: Northern and Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression
No correlation is shown between humidity and latitude for the Norther and Southern Hemispheres and no strong relationship between cloudiness and latitude for both the Northern and Southern Hemispheres.


In [1]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'citipy'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Get weather data
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
process_record = 0
set_counter = 1
cities_weather = []

for city in cities:
    
     # Increase counter by one 
    process_record += 1
    print(f"Processing Record {process_record} of Set {set_counter} | {city}")

    if(process_record==50):
        set_counter+=1
        process_record=0
    
    query_url = f"{url}&appid={weather_api_key}&q={city}&units={units}"

    try:
        response = requests.get(query_url).json()
        cities_weather.append({'City':city,
                               'Lat':response['coord']['lat'],
                               'Lng':response['coord']['lon'],
                               'Max Temp':response['main']['temp_max'],
                               'Humidity':response['main']['humidity'],
                               'Cloudiness':response['clouds']['all'],
                               'Wind Speed':response['wind']['speed'],
                               'Country':response['sys']['country'],
                               'Date':response['dt']})
    except:
        print('City not found. Skipping...')
        pass
        
    
print('-----------------------------')
print('Data Retrieval Complete')      
print('-----------------------------')

In [ ]:
cities = pd.DataFrame(cities_weather)
cities.head()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
cities.count()

In [ ]:
# Display the DataFrame
cities = pd.DataFrame(cities_weather)
cities

In [ ]:
# Export the data to csv
cities.to_csv(output_data_file)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# Display a statistical overview of the DataFrame 
cities.describe()

In [ ]:
# Cities that have humidity > 100%.
indices = cities.index[weather_data_df['Humidity'] > 100].tolist()
len(indices)

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = cities.drop(indices, inplace=False)
clean_city_data.head()

In [ ]:
# Export the City_Data into a csv
clean_city_data.to_csv("../output_data/clean_city_data.csv")

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Latitude Vs Temperature Scatter Plot 
plt.scatter(clean_city_data['Lat'],clean_city_data['Max Temp'],facecolors = "magenta", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs. Max Temperature ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Temperature.png")

plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Latitude vs. Humidity Scatter Plot 
plt.scatter(clean_city_data["Lat"],clean_city_data["Humidity"],facecolors = "cyan", edgecolors ="black",marker ="o")
plt.title(f'City Latitude vs. Humidity ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Humidity.png")

plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Latitude vs. Cloudiness Scatter Plot - **ADD CLEAN DATAFRAME?
plt.scatter(clean_city_data['Lat'],clean_city_data['Cloudiness'],facecolors = "plum", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs. Cloudiness ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Cloudiness.png")

plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Latitude vs. Wind Speed Scatter Plot 
plt.scatter(clean_city_data["Lat"],clean_city_data["Wind Speed"],facecolors = "steelblue", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs. Wind Speed ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_WindSpeed.png")

plt.show()

## Linear Regression

In [ ]:
def plot_linregress(X, y, title):
    print(f"The r-squared is: {round(st.pearsonr(X, y)[0],2)}")
    (slope, intercept, rvalue, pvalue, stderr) = linregress(X, y)
    regress_values = X * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "X + " + str(round(intercept,2))
    plt.scatter(X, y)
    plt.plot(X,regress_values,"r-")
    plt.title(title + '\n' + line_eq)
    plt.xlabel(X.name)
    plt.ylabel(y.name)


In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_hemisphere_df = clean_city_data.loc[clean_city_data['Lat'] >= 0]
print('northern_hemisphere_df.shape', northern_hemisphere_df.shape)

southern_hemisphere_df = clean_city_data.loc[clean_city_data['Lat'] < 0]
print('southern_hemisphere_df.shape', southern_hemisphere_df.shape)

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plot_linregress(northern_hemisphere_df['Lat'], northern_hemisphere_df['Max Temp'], 'Northern Hemisphere - Max Temp vs. Latitude Linear Regression')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plot_linregress(southern_hemisphere_df['Lat'], southern_hemisphere_df['Max Temp'], 'Southern Hemisphere -Max Temp vs. Latitude Linear Regression')

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plot_linregress(northern_hemisphere_df['Lat'], northern_hemisphere_df['Humidity'], 'Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plot_linregress(southern_hemisphere_df['Lat'], southern_hemisphere_df['Humidity'], 'Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression')

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plot_linregress(northern_hemisphere_df['Lat'], northern_hemisphere_df['Cloudiness'], 'Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plot_linregress(southern_hemisphere_df['Lat'], southern_hemisphere_df['Cloudiness'], 'Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression')

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plot_linregress(northern_hemisphere_df['Lat'], northern_hemisphere_df['Wind Speed'], 'Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plot_linregress(southern_hemisphere_df['Lat'], southern_hemisphere_df['Wind Speed'], 'Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression')